# Agentic RAG

이번 챕터에서는 문서 검색을 통해 최신 정보에 접근하여 검색 결과를 가지고 답변을 생성하는 에이전트를 만들어 보겠습니다.

질문에 따라 문서를 검색하여 답변하거나, 인터넷 검색 도구를 활용하여 답변하는 에이전트를 만들어 보겠습니다.

**참고**

- RAG 를 수행하되, Agent 를 활용하여 RAG 를 수행한다면 이를 **Agentic RAG** 라고 부릅니다.

## 도구(Tools)

Agent 가 활용할 도구를 정의하여 Agent 가 추론(reasoning)을 수행할 때 활용하도록 만들 수 있습니다.

Tavily Search 는 그 중 대표적인 **검색 도구** 입니다. 검색을 통해 최신 정보에 접근하여 검색 결과를 가지고 답변을 생성할 수 있습니다. 도구는 이처럼 검색 도구 뿐만아니라 Python 코드를 실행할 수 있는 도구, 직접 정의한 함수를 실행하는 도구 등 다양한 종류와 방법론을 제공합니다.


### 웹 검색도구: Tavily Search

LangChain에는 Tavily 검색 엔진을 도구로 쉽게 사용할 수 있는 내장 도구가 있습니다.

Tavily Search 를 사용하기 위해서는 API KEY를 발급 받아야 합니다.

- [Tavily Search API 발급받기](https://app.tavily.com/sign-in)

발급 받은 API KEY 를 다음과 같이 환경변수에 등록 합니다.

`.env` 파일에 다음과 같이 등록합니다.

- `TAVILY_API_KEY=발급 받은 Tavily API KEY 입력`

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Agents")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agents


In [3]:
# TavilySearchResults 클래스를 langchain_community.tools.tavily_search 모듈에서 가져옵니다.
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=6)

`search.invoke` 함수는 주어진 문자열에 대한 검색을 실행합니다.

`invoke()` 함수에 검색하고 싶은 검색어를 넣어 검색을 수행합니다.


In [4]:
# 검색 결과를 가져옵니다.
search.invoke("판교 카카오 프렌즈샵 아지트점의 전화번호는 무엇인가요?")

[{'url': 'https://blog.naver.com/PostView.naver?blogId=amour1115&logNo=223386213560',
  'content': '카카오 프렌즈 판교 아지트점. . 연중무휴 12:00-21:00. 토, 일 12:00-20:00. . 경기 성남시 분당구 판교역로 166. 카카오 판교 아지트 1층 카카오 프렌즈. 031-301-7225. 존재하지 않는 이미지입니다.'},
 {'url': 'https://kr.trip.com/moments/detail/seongnam-121856-121058702/',
  'content': '사진찍으면서 실내데이트 즐기고 싶다면 주목📢 판교역에서 꼭 가봐야할 아기자기 귀여운 핫플레이스 카카오 프렌즈 캐릭터들의 포토존이 가득한 대형 카카오 프렌즈를 판교점은 정말 특별해요! 📍카카오프렌즈 판교 아지트점 ️경기도 성남시 분당구 판교역로 166 카카오판교아지트 다른 카카오 ...'},
 {'url': 'https://m.blog.naver.com/kimsso333/223186761448',
  'content': '\u200b\n\u200b\n\u200b\n\u200b\n선물 후보로 끝까지 고민했던\n춘식이 얼음틀 ㅋㅋㅋ\n\u200b\n\u200b\n\u200b\n\u200b\n카카오프렌즈 판교점\n한쪽에는 콜프용품들도\n한가득 준비되어 있었고용\n\u200b\n\u200b\n\u200b\n\u200b\n카카오톡에 이모티콘 친구들도\nMD로 있어서 귀엽더라고요!\n\u200b\n\u200b\n\u200b\n\u200b\n귀여운 루피와\n일본에서 온 빤쮸토끼도\n매장 한켠에 있었어요 ㅎㅎㅎ\n\u200b\n\u200b\n\u200b\n\u200b\n보드게임도 있규여\n\u200b\n\u200b\n\u200b\n\u200b\n피규어와 비타민까지 -\n\u200b\n맘만먹으면\n카카오로 집꾸미기 가~넝~\n\u200b\n\u200b\n\u200b\n\u200b\n최고심X춘식이 쏘큐트\n\u20

### 문서 기반 문서 검색 도구: Retriever

우리의 데이터에 대해 조회를 수행할 retriever도 생성합니다.

**실습에 활용한 문서**

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`

_실습을 위해 다운로드 받은 파일을 `data` 폴더로 복사해 주시기 바랍니다_


이 코드는 웹 기반 문서 로더, 문서 분할기, 벡터 저장소, 그리고 OpenAI 임베딩을 사용하여 문서 검색 시스템을 구축합니다.

여기서는 PDF 문서를 `FAISS` DB 에 저장하고 조회하는 retriever 를 생성합니다.


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

이 함수는 `retriever` 객체의 `invoke()` 를 사용하여 사용자의 질문에 대한 가장 **관련성 높은 문서** 를 찾는 데 사용됩니다.


In [6]:
# 문서에서 관련성 높은 문서를 가져옵니다.
retriever.invoke("삼성전자가 개발한 생성형 AI 관련 내용을 문서에서 찾아줘")

[Document(metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 12}, page_content='SPRi AI Brief |  \n2023-12 월호\n10삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개\nn삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스 ’를 공개\nn삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로 , 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유KEY Contents\n£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스 , 온디바이스 작동 지원\nn삼성전자가 2023 년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스 ’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스 (Gauss) 의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며 , \n온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유\n∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며 , 생성 AI 모델을 다양한 제품에 \n단계적으로 탑재할 계획\nn삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는  \n이미지 모델의 3개 모델로 구성\n∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며 , 메일 작성, 문서 요약, 번역 업무의 \n처리를 지원\n∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이 (code.i)’ 는 대화형 인터페이스로 서비스를 제공하며 \n사내 소프트웨어 개발에 최적화\n∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 수 있

이제 우리가 검색을 수행할 인덱스를 채웠으므로, 이를 에이전트가 제대로 사용할 수 있는 도구로 쉽게 변환할 수 있습니다.


`create_retriever_tool` 함수로 `retriever` 를 도구로 변환합니다.

In [7]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

### Agent 가 사용할 도구 목록 정의

이제 두 가지를 모두 만들었으므로, Agent 가 사용할 도구 목록을 만들 수 있습니다.

`tools` 리스트는 `search`와 `retriever_tool`을 포함합니다. 

In [8]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

## Agent 생성

이제 도구를 정의했으니 에이전트를 생성할 수 있습니다. 

먼저, Agent 가 활용할 LLM을 정의하고, Agent 가 참고할 Prompt 를 정의합니다.

**참고**
- 멀티턴 대화를 지원하지 않는다면 "chat_history" 를 제거해도 좋습니다.

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

다음으로는 Tool Calling Agent 를 생성합니다.

In [10]:
from langchain.agents import create_tool_calling_agent

# tool calling agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

마지막으로, 생성한 `agent` 를 실행하는 `AgentExecutor` 를 생성합니다.

**참고**

- `verbose=False` 로 설정하여 중간 단계 출력을 생략하였습니다.

In [11]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

## 에이전트 실행하기

이제 몇 가지 질의에 대해 에이전트를 실행할 수 있습니다!

현재 이러한 모든 질의는 **상태(Stateless) 가 없는** 질의입니다(이전 상호작용을 기억하지 않습니다).


`agent_executor` 객체의 `invoke` 메소드는 딕셔너리 형태의 인자를 받아 처리합니다. 이 예제에서는 `input` 키에 `hi!` 값을 할당한 딕셔너리를 인자로 전달하고 있습니다. 이는 일반적으로 AI 에이전트, 함수 실행기, 또는 명령 처리기 등의 객체에서 입력을 처리하기 위해 사용됩니다.


In [12]:
from langchain_teddynote.messages import AgentStreamParser

# 각 단계별 출력을 위한 파서 생성
agent_stream_parser = AgentStreamParser()

In [13]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "2024년 프로야구 플레이오프 진출한 5개 팀을 검색하여 알려주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 2024년 프로야구 플레이오프 진출 팀
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`



[관찰 내용]
Observation: [{'url': 'https://www.yna.co.kr/view/GYH20241003000600044', 'content': '4 days ago · ... 팀의 뒤집기 마법을 선사하고 준플레이오프(준PO·5전 3승제)에 진출했다. kt는 3일 서울 잠실구장에서 열린 2024 신한 SOL뱅크 KBO 포스트시즌 와일드\xa0...'}, {'url': 'https://www.hani.co.kr/arti/sports/baseball/1159732.html', 'content': 'Sep 25, 2024 · 준플레이오프 승자는 플레이오프에 진출해 정규리그 2위 삼성 라이온즈와 맞붙는다. 5전3선승제로 진행되고 1·2·5차전은 10월13일 대구 삼성라이온즈파크\xa0...'}, {'url': 'https://m.blog.naver.com/qhadldhaus98/223605904423', 'content': '4 days ago · ... 프로야구 포스트시즌에 진출한 팀들은. KIA 타이거즈 삼성 라이온즈 LG 트윈스. 두산 베어스 KT위즈입니다. 출처 KBO. 출처 두산베어스. 출처 두산베어스.'}, {'url': 'https://www.chosun.com/sports/baseball/2024/09/25/R6MITAGAEBBC7JDIQOB7KW3OUY/', 'content': 'Sep 25, 2024 · 4위의 홈 경기장에서 최대 2경기가 열린다. 4위는 한 경기에서 승리 또는 무승부만 올려도 준플레이오프에 진출한다. 5위는 2연승을 거둬야 준플레이오프\xa0...'}, {'url': 'https://namu.wiki/w/KBO%20%ED%94%8C

`agent_executor` 객체의 `invoke` 메소드를 사용하여, 질문을 입력으로 제공합니다.


In [17]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "삼성전자가 자체 개발한 생성형 AI 관련된 정보를 문서에서 찾아주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 삼성전자 생성형 AI
Log: 
Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`



[관찰 내용]
Observation: SPRi AI Brief |  
2023-12 월호
10삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개
n삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스 ’를 공개
n삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로 , 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유KEY Contents
£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스 , 온디바이스 작동 지원
n삼성전자가 2023 년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스 ’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스 (Gauss) 의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며 , 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며 , 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는  
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며 , 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이 (code.i)’ 는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원

## 이전 대화내용 기억하는 Agent

이전의 대화내용을 기억하기 위해서는 `RunnableWithMessageHistory` 를 사용하여 `AgentExecutor` 를 감싸줍니다.

`RunnableWithMessageHistory` 에 대한 자세한 내용은 아래 링크를 참고해 주세요.

**참고**
- [RunnableWithMessageHistory](https://wikidocs.net/254682)

In [21]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

In [22]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.stream(
    {"input": "삼성전자가 개발한 생성형 AI 관련된 정보를 문서에서 찾아주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 삼성전자 생성형 AI
Log: 
Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`



[관찰 내용]
Observation: SPRi AI Brief |  
2023-12 월호
10삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개
n삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스 ’를 공개
n삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로 , 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유KEY Contents
£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스 , 온디바이스 작동 지원
n삼성전자가 2023 년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스 ’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스 (Gauss) 의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며 , 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며 , 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는  
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며 , 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이 (code.i)’ 는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원

In [23]:
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
Here is the translation of the previous response into English:

1. **Samsung Gauss**: On November 8, 2023, Samsung Electronics unveiled its generative AI model called 'Samsung Gauss' at the 'Samsung AI Forum 2023'. This model consists of three components: language, code, and image models, and is designed to operate on-device. This has the advantage of ensuring that user information is not at risk of being leaked externally.

2. **Model Composition**:
   - **Language Model**: Composed of various models for cloud and on-device use, it supports tasks such as email writing, document summarization, and translation.
   - **Code Model**: Through the AI coding assistant 'code.i', it provides services via an interactive interface, optimized for in-house software development.
   - **Image Model**: Capable of generating creative images and modifying existing ones, it also supports converting low-resolution images to high-resolution.

3. **Future Plans**: Samsung Electronics plans to gradu

## Agent 템플릿

다음은 전체 템플릿 코드 입니다.


In [1]:
# 필요한 모듈 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_teddynote.messages import AgentStreamParser

########## 1. 도구를 정의합니다 ##########

### 1-1. Search 도구 ###
# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=6)

### 1-2. PDF 문서 검색 도구 (Retriever) ###
# PDF 파일 로드. 파일의 경로 입력
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

### 1-3. tools 리스트에 도구 목록을 추가합니다 ###
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

########## 2. LLM 을 정의합니다 ##########
# LLM 모델을 생성합니다.
llm = ChatOpenAI(model="gpt-4o", temperature=0)

########## 3. Prompt 를 정의합니다 ##########

# Prompt 를 정의합니다 - 이 부분을 수정할 수 있습니다!
# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

########## 4. Agent 를 정의합니다 ##########

# 에이전트를 생성합니다.
# llm, tools, prompt를 인자로 사용합니다.
agent = create_tool_calling_agent(llm, tools, prompt)

########## 5. AgentExecutor 를 정의합니다 ##########

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정합니다.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

########## 6. 채팅 기록을 수행하는 메모리를 추가합니다. ##########

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

########## 7. Agent 파서를 정의합니다. ##########
agent_stream_parser = AgentStreamParser()

In [2]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "구글이 앤스로픽에 투자한 금액을 문서에서 찾아줘"},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 구글 앤스로픽 투자 금액
Log: 
Invoking: `pdf_search` with `{'query': '구글 앤스로픽 투자 금액'}`



[관찰 내용]
Observation: 1. 정책/법제  
2. 기업/산업 
3. 기술/연구 
 4. 인력/교육
구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 
n 구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은 
구글과 클라우드 서비스 사용 계약도 체결
n 3대 클라우드 사업자인 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인 
앤스로픽 및 오픈AI와 협력을 확대하는 추세
KEY Contents
£ 구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공
n 구글이 2023년 10월 27일 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 
달러를 우선 투자하고 향후 15억 달러를 추가로 투자할 방침
∙구글은 2023년 2월 앤스로픽에 이미 5억 5,000만 달러를 투자한 바 있으며, 아마존도 지난 9월 
앤스로픽에 최대 40억 달러의 투자 계획을 공개
∙한편, 2023년 11월 8일 블룸버그 보도에 따르면 앤스로픽은 구글의 클라우드 서비스 사용을 위해 
4년간 30억 달러 규모의 계약을 체결
∙오픈AI 창업자 그룹의 일원이었던 다리오(Dario Amodei)와 다니엘라 아모데이(Daniela Amodei) 
남매가 2021년 설립한 앤스로픽은 챗GPT의 대항마 ‘클로드(Claude)’ LLM을 개발
n 아마존과 구글의 앤스로픽 투자에 앞서, 마이크로소프트는 차세대 AI 모델의 대표 주자인 오픈
AI와 협력을 확대
∙마이크로소프트는 오픈AI에 앞서 투자한 30억 달러에 더해 2023년 1월 추가로 100억 달러를 
투자하기로 하면서 오픈AI의 지분 49%를 확보했으며, 오픈AI는 마이크로소프트의 애저(Azure) 
클라우드 플랫폼을 사용해 AI 모델을 훈련
£ 

In [3]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요"},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
Google has agreed to invest up to $2 billion in Anthropic, with an initial investment of $500 million. They plan to invest an additional $1.5 billion in the future.


In [6]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {
        "input": "2024년 프로야구 플레이오프 진출 5개팀을 검색해서 알려주세요. 한글로 답변하세요"
    },
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 2024 KBO 리그 플레이오프 진출팀
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024 KBO 리그 플레이오프 진출팀'}`



[관찰 내용]
Observation: [{'url': 'https://www.gosusports.com/news/baseball/kbo-news-2024-kbo-season-preview/196180/', 'content': "[고수스포츠] 2024 kbo 리그 시즌 프리뷰 ... 플레이오프 진출을 이뤄냈다. nc의 선전에는 '특급 외인' 페디의 활약이 주효했다. 페디는 평균자책점(era 2.00)다승(20승), 탈삼진(209개) 등 3가지 부문에서 모두 1위를 차지하며 kbo리그를 평정했다."}, {'url': 'https://namu.wiki/w/2024+신한+SOL+Bank+KBO+리그', 'content': '당초 입찰이 유력하다고 알려졌던 쿠팡플레이는 온라인 영상 중계권을 독점한 다음 모든 온라인 중계를 유료로 돌릴 ... kbo는 지애드스포츠를 2024-2026년 kbo 리그 해외 중계권 사업자로 선정했다. ... kbs n sports의 v-리그 여자부 플레이오프 3차전 중계로 7회초 ...'}, {'url': 'https://ko.wikipedia.org/wiki/2024년_KBO_리그', 'content': '2024 신한은행 SOL Bank KBO 리그; 주관: KBO: 날짜: 3월 23일 ~ 10월 1일: 참가팀 수: 10: 정규 시즌; 포스트 시즌; 준플레이오프: kt 0 - 0 LG: 플레이오프'}, {'url': 'https://www.wikitree.co.kr/articles/988195', 'content': "이 그래픽을 통해 팬들은 '2024 준플레이오프 일정 ... 지난 3일 오후 서울 송파구 잠실야구장에서 열린 프로야구 '2024 신한 sol 

In [7]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 SNS 게시글 형태로 100자 내외로 작성하세요."},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
2024 KBO 리그 플레이오프 진출팀: KIA 타이거즈, 삼성 라이온즈, LG 트윈스, 두산 베어스, KT 위즈! 치열한 경기가 기대됩니다! #KBO #플레이오프


In [9]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변에 한국 시리즈 일정을 추가하세요."},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 2024 KBO 한국 시리즈 일정
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024 KBO 한국 시리즈 일정'}`



[관찰 내용]
Observation: [{'url': 'https://ko.wikipedia.org/wiki/2024년_한국시리즈', 'content': '2024 신한 SOL Bank KBO 한국시리즈는 2024년 10월 21일부터 10월 29일까지 펼쳐질 예정인 2024년 KBO 리그의 챔피언 결정전이다. 대진은 2024년 KBO 리그 정규 시즌 1위 팀으로 2017년 한국시리즈 이후 7년만에 한국시리즈 직행에 성공하며 통합 우승에 도전하는 KIA 타이거즈 와 ...'}, {'url': 'https://www.wikiwand.com/ko/articles/2024년_한국시리즈', 'content': '2024 신한은행 SOL KBO 한국시리즈 는 2024년 10월 20일 부터 10월 28일 까지 펼쳐질 예정인 2024년 KBO 리그 의 챔피언 결정전이다. 대진은 2024년 KBO 리그 정규 시즌 1위 팀으로 2017년 한국시리즈 이후 7년만에 한국시리즈 직행에 성공하며 통합 우승에 도전하는 KIA ...'}, {'url': 'https://namu.wiki/w/2024+신한+SOL+Bank+KBO+한국시리즈', 'content': '예매 일정 2.2. 각 구장 티켓 가격 / 좌석배치도 ... 2024 신한은행 SOL KBO 한국시리즈 3차전 2024 Shinhan Bank SOL KBO KOREAN SERIES Game 3. 10월 24일 18:30 ~ 00:00 (0시간 0분)'}, {'url': 'https://namu.wiki/w/2024년+한국시리즈', 'content': '2024년 한국시리즈. 2025년 한국시리즈. 2026년 한국시리즈. [A]: 삼성 라이온즈 가 전·후반기를